# Padrão Strategy usando classe

**Definição**. Padrão que:
1. define uma família de algoritmos,
2. encapsuca dada um,
3. torna-os intercambiáveis.

## Ator e Objeto

In [11]:
from collections import namedtuple


Cliente = namedtuple('Cliente', 'nome fidelidade')

class ProdutoNoCarrinho:
    def __init__(self, nome, quantidade, preco) -> None:
        self.nome = nome
        self.quantidade = quantidade
        self.preco = preco

    def total(self):
        return self.preco * self.quantidade

## Contexto

In [12]:
class Pedido:
    def __init__(self, cliente, carrinho, promocao=None) -> None:
        self.cliente = cliente
        self.carrinho: list[ProdutoNoCarrinho] = carrinho
        self.promocao = promocao
        self.desconto = 0.0

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.carrinho)
        return self.__total

    def desconte(self):
        self.desconte = self.promocao.desconto(self)

    def total_considerando_descontos(self):
        if self.promocao is None:
            return self.total()
        self.desconte()
        return self.total() - self.desconto

    def __repr__(self) -> str:
        linhas = [
            ("Subtotal", f"R$ {self.total():10.2f}"),
            (" Desconto", f"- R$ {self.desconto:10.2f}"),
            (" Subtotal", f"R$ {self.total_considerando_descontos():10.2f}"),
        ]
        return "<" + '\n'.join(f"{tipo}: {valor}" for tipo,valor in linhas) + ">"

## Interface da estratégia

In [13]:
import abc

class Promocao(abc.ABC):
    @abc.abstractmethod
    def desconto(self, pedido: Pedido) -> float: pass

## Estratégias

In [14]:
class FidelidadePromo(Promocao):
    def desconto(self, pedido: Pedido) -> float:
        return pedido.total()


class CombinadosPromo(Promocao):
    def desconto(self, pedido: Pedido) -> float:
        itens_elegiveis = [item for item in pedido.carrinho if item.quantidade >= 20]
        return sum(item.total() for item in itens_elegiveis) / 10


class PedidoGrandePromo(Promocao):
    def desconto(self, pedido: Pedido) -> float:
        if len({item.nome for item in pedido.carrinho}) >= 10:
            return pedido.total() * 0.07
        return 0